<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/acme.jpeg width=100 align="left">

# Tokenize Data

There are organizations that care more about the irreversibility of the tokenized data and not so much about preserving the original data format.

Tokenization replaces sensitive data with unique values (tokens) that are unrelated to the origina value.  Those tokens cannot risk exposing the plaintext satifing the PCI-DSS guidance.

<img src="images/vault-tokenization-1.png">

**Characteristics of tokenization:** 

* non-reversible
* integrated metadata:  supports metdata for identifying data type and purpose
* extreme scale and performance 


**Prerequisites:**
* Vault Enterprise binary with Advanced Data Protection
* Vault License

## Setup

In [6]:
/bin/rm -rf $HOME/demos/vault_cluster/vrd/data/*

In [148]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true
export VAULT_ROOT_TOKEN=root

In [16]:
which vault
vault --version


/usr/local/bin/vault
Vault v1.6.0 (7ce0bd9691998e0443bc77e98b1e2a4ab1e965d4)


\
**EXECUTE the following command in SEPARATE window**

```
VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 evault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional
```
OR   
`evrd1`   

## (Optional)  Apply the license

In [151]:
curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_ROOT_TOKEN" \
    --request PUT \
    --data @/Users/tio/Documents/vault_license.json \
    $VAULT_ADDR/v1/sys/license

In [152]:
vault status

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.6.1+ent
Storage Type    inmem_transactional_ha
Cluster Name    vault-cluster-60e9d409
Cluster ID      5c25711e-51db-0b79-f3d5-4e9a0080828e
HA Enabled      true
HA Cluster      https://127.0.0.1:8201
HA Mode         active
Last WAL        35


##### Notice Initialized (=true) and Sealed (=false) status 

## Enable the Transform secrets engine

In [153]:
vault secrets enable transform

Success! Enabled the transform secrets engine at: transform/


## Roles, Transformations, Templates and Alphabets

Create a role `mobile-pay` which is attached to `credit-card` transformation.  The tokenized value has a fixed maximum time-to-live of 24 hours.  

<img src=images/vault-tokenization-2.png >


### Create a Role

In [189]:
vault write transform/role/mobile-pay transformations=credit-card

Success! Data written to: transform/role/mobile-pay


### Specify the transformation

In [194]:
vault write transform/transformation/credit-card \
  type=tokenization \
  max_ttl=24h \
  allowed_roles=mobile-pay

Success! Data written to: transform/transformation/credit-card


### Transform it
After the secrets engine is configured and a user/machine has a Vault token, it can use this secret engine to encode and decode input values

In [196]:
vault write transform/encode/mobile-pay value=1111-2222-3333-4444 \
  ttl=8h \
  metadata="type=Amex" \
  metadata="organization=HashiCorp"

Key              Value
---              -----
encoded_value    eRwUjS2L9dvCTxKCmrwRas1tfjaAhvcvMFpgfRrX4uAwofSmpyNbje3aX3mXXJvs3zxwVsxSE


### Decode some input value using the /decode endpoint with a named role:

In [197]:
export MY_ENCODED_CCN="eRwUjS2L9dvCTxKCmrwRas1tfjaAhvcvMFpgfRrX4uAwofSmpyNbje3aX3mXXJvs3zxwVsxSE"
vault write transform/metadata/mobile-pay value=$MY_ENCODED_CCN

Key                Value
---                -----
expiration_time    2020-12-30 19:40:57.631628 +0000 UTC
metadata           map[organization:HashiCorp type:Amex]


In [198]:
vault write transform/validate/mobile-pay value=$MY_ENCODED_CCN

Key      Value
---      -----
valid    true


In [199]:
vault write transform/tokenized/mobile-pay value=$MY_ENCODED_CCN

Key          Value
---          -----
tokenized    false


In [200]:
vault write transform/decode/mobile-pay value=$MY_ENCODED_CCN

Key              Value
---              -----
decoded_value    1111-2222-3333-4444


### Setup external token storage
Tokenization is a stateful procedure to facilitate mapping between tokens and various cryptographic values.  This could put a lot of load on the Vault's storage backend.  You have an option to use external storage to presist data for tokenization tranformation.

<img src="images/vault-tokenization-3.png">  

To demonstrate, run a PostgreSQL database. 
Create a new transformation, named "passport", which uses this PostgreSQL as its storage.


&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">